# Exercise 2

## 2 Linear Activation Function

<div style="color: green; font-weight: bold">
The solution that we have for the question is the same: In our approach we also showed that a multi layer network is the same as a 1-layer network when using a linear activation function by repeating the operation recursively.
</div>

## 3 Programming a neural network

In this exercise a simple Multi-Layer Perceptron classifer is implemented using numpy. The python code below defines an MLP class with ReLU activations in the hidden layers and softmax output.

In [2]:
import numpy as np
from sklearn import datasets

In [3]:
####################################

class ReLULayer(object):
    def forward(self, input):
        # remember the input for later backpropagation
        self.input = input
        # return the ReLU of the input
        # ReLu: returns the input if it's positive otherwise returns 0
        relu = np.maximum(0, input)
        return relu

    def backward(self, upstream_gradient):
        # compute the derivative of ReLU from upstream_gradient and the stored input
        # ( self.input > 0) is a binary mask, only passes input elements that had a positive value
        downstream_gradient = upstream_gradient * ( self.input > 0)
        return downstream_gradient

    def update(self, learning_rate):
        pass # ReLU is parameter-free

OutputLayer class:
- the forward method computes the softmax of the input
- the backward method computes the gradient of the loss with respect to the stored inputs. The gradient is computed by substracting 1 from the predicted posterior probabilities at the true label indices and divides them by the batch size. 
    (Zlk -1 if k = Yi*,
    Zlk otherwise)

In [4]:
####################################

class OutputLayer(object):
    def __init__(self, n_classes):
        self.n_classes = n_classes

    def forward(self, input):
        # remember the input for later backpropagation
        self.input = input
        # return the softmax of the input
        input_exp = np.exp(input)
        softmax = input_exp / np.sum(input_exp, axis = 1, keepdims=True) # your code here
        return softmax

    def backward(self, predicted_posteriors, true_labels):
        # return the loss derivative with respect to the stored inputs
        # use cross-entropy loss and the chain rule for softmax
        batch_size = predicted_posteriors.shape[0]
        downstream_gradient = predicted_posteriors.copy()
        downstream_gradient[np.arange(batch_size), true_labels] -= 1
        downstream_gradient /= batch_size
        return downstream_gradient

    def update(self, learning_rate):
        pass # softmax is parameter-free

LinearLayer class:
- the forward method stores the input for later use and the preactivations are computed by taking the dot product of the input and weight and then addind the bias.
    (Zin.B + b)
- the backward method calculates the gradients for weights and biases using the upstream gradient from the next layer. The gradient for the biases is the sum of the upstream gradient along the first axis. The gradient of the weight is the dot product of the input transposed and the upstream gradient. The downstream gradient is the dot product of the upstream gradient and the transposed weights.

In [5]:
####################################

class LinearLayer(object):
    def __init__(self, n_inputs, n_outputs):
        self.n_inputs  = n_inputs
        self.n_outputs = n_outputs
        # randomly initialize weights and intercepts
        # Weight matrix
        self.B = np.random.normal(size = (n_inputs, n_outputs))
        # Bias vector
        self.b = np.random.normal(size = (1, n_outputs))

    def forward(self, input):
        # remember the input for later backpropagation
        self.input = input
        # compute the scalar product of input and weights
        # (these are the preactivations for the subsequent non-linear layer)
        preactivations = np.dot(input, self.B) + self.b
        return preactivations

    def backward(self, upstream_gradient):
        # compute the derivative of the weights from
        # upstream_gradient and the stored input
        # Gradient of bias
        self.grad_b = np.sum(upstream_gradient, axis=0)
        # Gradient of weights
        self.grad_B = np.dot(self.input.T, upstream_gradient)
        # compute the downstream gradient to be passed to the preceding layer
        # Chain rule
        downstream_gradient = np.dot(upstream_gradient, self.B.T)
        return downstream_gradient

    def update(self, learning_rate):
        # update the weights by batch gradient descent
        self.B = self.B - learning_rate * self.grad_B
        self.b = self.b - learning_rate * self.grad_b

MLP class:
- the forward method reshapes input X to have dimensions (batch_size, n_features) and then iterates through each layer and computes forward pass by calling the forward method of each layer. The result of each layer is the input of the next layer.
- the backward method performs the backpropagation by first calling the backward method of the last layer with predicted posteriors and true classes to get the upstream gradient. It iterates in reverse order through the layers and calls the backward method of the layers except the last one. The upstream gradient is updated at each layer and passed to the other layer until the backpropagation is completed.

In [6]:
####################################

class MLP(object):
    def __init__(self, n_features, layer_sizes):
        # constuct a multi-layer perceptron
        # with ReLU activation in the hidden layers and softmax output
        # (i.e. it predicts the posterior probability of a classification problem)
        #
        # n_features: number of inputs
        # len(layer_size): number of layers
        # layer_size[k]: number of neurons in layer k
        # (specifically: layer_sizes[-1] is the number of classes)
        self.n_layers = len(layer_sizes)
        self.layers   = []

        # create interior layers (linear + ReLU)
        n_in = n_features
        for n_out in layer_sizes[:-1]:
            self.layers.append(LinearLayer(n_in, n_out))
            self.layers.append(ReLULayer())
            n_in = n_out

        # create last linear layer + output layer
        n_out = layer_sizes[-1]
        self.layers.append(LinearLayer(n_in, n_out))
        self.layers.append(OutputLayer(n_out))

    def forward(self, X):
        # X is a mini-batch of instances
        batch_size = X.shape[0]
        # flatten the other dimensions of X (in case instances are images)
        X = X.reshape(batch_size, -1)

        # compute the forward pass
        # (implicitly stores internal activations for later backpropagation)
        result = X
        for layer in self.layers:
            result = layer.forward(result)
        return result

    def backward(self, predicted_posteriors, true_classes):
        # perform backpropagation w.r.t. the prediction for the latest mini-batch X
        upstream_gradient = self.layers[-1].backward(predicted_posteriors, true_classes)
        for layer in reversed(self.layers[:-1]):
            upstream_gradient = layer.backward(upstream_gradient)
    
    def update(self, X, Y, learning_rate):
        posteriors = self.forward(X)
        self.backward(posteriors, Y)
        for layer in self.layers:
            layer.update(learning_rate)

    def train(self, x, y, n_epochs, batch_size, learning_rate):
        N = len(x)
        n_batches = N // batch_size
        for i in range(n_epochs):
            # print("Epoch", i)
            # reorder data for every epoch
            # (i.e. sample mini-batches without replacement)
            permutation = np.random.permutation(N)

            for batch in range(n_batches):
                # create mini-batch
                start = batch * batch_size
                x_batch = x[permutation[start:start+batch_size]]
                y_batch = y[permutation[start:start+batch_size]]

                # perform one forward and backward pass and update network parameters
                self.update(x_batch, y_batch, learning_rate)

In [27]:
##################################

if __name__=="__main__":

    # set training/test set size
    N = 2000

    # create training and test data
    X_train, Y_train = datasets.make_moons(N, noise=0.05)
    X_test,  Y_test  = datasets.make_moons(N, noise=0.05)
    n_features = 2
    n_classes  = 2

    # standardize features to be in [-1, 1]
    offset  = X_train.min(axis=0)
    scaling = X_train.max(axis=0) - offset
    X_train = ((X_train - offset) / scaling - 0.5) * 2.0
    X_test  = ((X_test  - offset) / scaling - 0.5) * 2.0

    # set hyperparameters (play with these!)
    layer_sizes = [5, 5, n_classes]
    n_epochs = 5
    batch_size = 200
    learning_rate = 0.05

    # create network
    network = MLP(n_features, layer_sizes)

    # train
    network.train(X_train, Y_train, n_epochs, batch_size, learning_rate)

    # test
    predicted_posteriors = network.forward(X_test)
    # determine class predictions from posteriors by winner-takes-all rule
    predicted_classes = np.argmax(predicted_posteriors, axis=1)
    # compute and output the error rate of predicted_classes
    error_rate = 1-np.sum(predicted_classes == Y_test) / len(Y_test)
    print("error rate:", error_rate)

error rate: 0.1965


Compare the validation errors on the dataset generated by the make_moons() function for the following four networks:
- MLP(n_features, [2, 2, n_classes]) -> 2 hidden layers, each with 2 neurons.
- MLP(n_features, [3, 3, n_classes]) -> 2 hidden layers, each with 3 neurons.
- MLP(n_features, [5, 5, n_classes]) -> 2 hidden layers, each with 5 neurons.
- MLP(n_features, [30, 30, n_classes]) -> 2 hidden layers, each with 30 neurons.

In [29]:
##################################

if __name__=="__main__":

    # set training/test set size
    N = 2000

    # create training and test data
    X_train, Y_train = datasets.make_moons(N, noise=0.05)
    X_test,  Y_test  = datasets.make_moons(N, noise=0.05)
    n_features = 2
    n_classes  = 2

    # standardize features to be in [-1, 1]
    offset  = X_train.min(axis=0)
    scaling = X_train.max(axis=0) - offset
    X_train = ((X_train - offset) / scaling - 0.5) * 2.0
    X_test  = ((X_test  - offset) / scaling - 0.5) * 2.0

    # set hyperparameters (play with these!)
    layer_sizes = [2, 2, n_classes]  # MLP(n_features, [2, 2, n_classes])
    layer_sizes2 = [3, 3, n_classes]  # MLP(n_features, [3, 3, n_classes])
    layer_sizes3 = [5, 5, n_classes]  # MLP(n_features, [5, 5, n_classes])
    layer_sizes4 = [30, 30, n_classes]  # MLP(n_features, [30, 30, n_classes])

    n_epochs = 5
    batch_size = 200
    learning_rate = 0.05

    # create network
    network = MLP(n_features, layer_sizes)
    network2 = MLP(n_features, layer_sizes2)
    network3 = MLP(n_features, layer_sizes3)
    network4 = MLP(n_features, layer_sizes4)

    # train
    network.train(X_train, Y_train, n_epochs, batch_size, learning_rate)
    network2.train(X_train, Y_train, n_epochs, batch_size, learning_rate)
    network3.train(X_train, Y_train, n_epochs, batch_size, learning_rate)
    network4.train(X_train, Y_train, n_epochs, batch_size, learning_rate)

    # test
    predicted_posteriors = network.forward(X_test)
    predicted_posteriors2 = network2.forward(X_test)
    predicted_posteriors3 = network3.forward(X_test)
    predicted_posteriors4 = network4.forward(X_test)
    
    # determine class predictions from posteriors by winner-takes-all rule
    predicted_classes = np.argmax(predicted_posteriors, axis=1)
    predicted_classes2 = np.argmax(predicted_posteriors2, axis=1)
    predicted_classes3 = np.argmax(predicted_posteriors3, axis=1)
    predicted_classes4 = np.argmax(predicted_posteriors4, axis=1)
    
    # compute and output the error rate of predicted_classes
    error_rate = 1-np.sum(predicted_classes == Y_test) / len(Y_test)
    error_rate2 = 1-np.sum(predicted_classes2 == Y_test) / len(Y_test)
    error_rate3 = 1-np.sum(predicted_classes3 == Y_test) / len(Y_test)
    error_rate4 = 1-np.sum(predicted_classes4 == Y_test) / len(Y_test)    
    
    # print error rates
    print("Validation Errors:")
    print("MLP([2, 2, n_classes]):", error_rate)
    print("MLP([3, 3, n_classes]):", error_rate2)
    print("MLP([5, 5, n_classes]):", error_rate3)
    print("MLP([30, 30, n_classes]):", error_rate4)

Validation Errors:
MLP([2, 2, n_classes]): 0.16000000000000003
MLP([3, 3, n_classes]): 0.20199999999999996
MLP([5, 5, n_classes]): 0.10999999999999999
MLP([30, 30, n_classes]): 0.027000000000000024


Comparing the validation errors we can see that the network with 30 neurons has the best performance. The performance does not increase according to the number of neurons, since the network with 2 neurons seems to have a lower error rate than network with 3 neurons.

This behaviour could arise from a dataset whose datapoints show a somewhat linearly separable trend (have a linear tendency maybe with some errors) on a broader level, but whose true underlying classification decision boundary lies in a deeply transfored non-linear space. Hence, when only a few linear transformations have been done (2\*2), the prediction error is less than with intermediate nr of non-linear transformations (3\*3), which underfits the data. Lastly, the prediction error increases again with deeper non-linear transformations of the input space (> 5\*5).